# Delaunay-Watershed for data compression : Nuclei Data

In this notebook, we present how to compress label data in a multimaterial mesh and recompute the mask from the mesh and the seeds if necessary

## 1) Download nuclei data from Stardist 

In [1]:
from __future__ import print_function, unicode_literals, absolute_import, division
import numpy as np
from glob import glob
from tqdm import tqdm
from csbdeep.utils import Path, download_and_extract_zip_file

download_and_extract_zip_file(
    url       = 'https://github.com/stardist/stardist/releases/download/0.3.0/demo3D.zip',
    targetdir = '../data',
    verbose   = 1,)


X = sorted(glob('../data/train/images/*.tif'))
Y = sorted(glob('../data/train/masks/*.tif'))
assert all(Path(x).name==Path(y).name for x,y in zip(X,Y))

Files found, nothing to download.


## 2) Compute multimaterial mesh with Delaunay Watershed

In [2]:
from dw3d import geometry_reconstruction_3d, reconstruct_mask_from_dict
import skimage.io as io 
from scipy.ndimage import zoom

def make_idx(n): 
    return(("000"+str(n))[-3:])

dist = 2
i = 2
labels = zoom(io.imread(Y[i]), [2,1,1,],order = 0)
image = zoom(io.imread(X[i]), [2,1,1,],order = 1)

DW = geometry_reconstruction_3d(labels,min_dist = dist, expansion_labels =0,mode='torch',print_info=False,original_image = image)
#v = DW.plot_in_napari(add_mesh=True)

## 3) Export segmentations for reconstruction. Visualize in napari

In [3]:
DW.export_segmentation("Dict_mask.npy")

In [4]:
labels = reconstruct_mask_from_dict("Dict_mask.npy")

In [5]:
import napari
v = napari.view_image(labels)
v.add_image(DW.labels)

<Image layer 'Image' at 0x7fe598c76ca0>

## 4) Print size saved : 

In [6]:
import sys
import numpy as np 
D = np.load("Dict_mask.npy",allow_pickle=True).item()
Dict_memory_footprint = D['Verts'].nbytes + D['Faces'].nbytes + D['seeds'].nbytes + D['image_shape'].nbytes
print("Size of Delaunay-Watershed data:",Dict_memory_footprint,"bytes")
print("Size of the labels:",labels.nbytes,"bytes")
print("Relative size:", Dict_memory_footprint/labels.nbytes)

Size of Delaunay-Watershed data: 963712 bytes
Size of the labels: 8388608 bytes
Relative size: 0.1148834228515625
